In [8]:
from pyspark.sql.functions import *
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
import csv
from pyspark.sql import HiveContext, SparkSession

In [3]:
conf=SparkConf().setAppName('Data Analysis Using SparkSQL')
sc=SparkContext(conf=conf)

In [6]:
#set the hive context
hc=HiveContext(sc)
hc.setConf("hive.metastore.uris","thrift://METASTORE:9083")

In [9]:
#Start spark SQL Session 
sparksession=SparkSession.builder.appName('Using SQL').config('"spark.some.config.option","some-value"').getOrCreate()

In [10]:
#read csv and infer the schema so that it can be used to create table in Hive
rcsv=sparksession.read.csv('Fake_data.csv',header=True,inferSchema=True)

In [11]:
#register table into Hive to query in it
rcsv.registerTempTable('tab')

In [12]:
#1.Birth Country with highest people
print('\n Birth Country with highest people')
ans=hc.sql("Select Birth_Country,count(Birth_Country) from tab group by Birth_Country order by count(Birth_Country)")
print(ans.show())


 Birth Country with highest people
+--------------------+--------------------+
|       Birth_Country|count(Birth_Country)|
+--------------------+--------------------+
|Svalbard & Jan Ma...|                  23|
|Holy See (Vatican...|                  23|
|             Ecuador|                  26|
|    Pitcairn Islands|                  27|
|             Romania|                  27|
|Syrian Arab Republic|                  27|
|           Singapore|                  28|
|French Southern T...|                  29|
|              Belize|                  29|
|            Malaysia|                  30|
|               Spain|                  30|
|            Barbados|                  31|
|             Bahamas|                  31|
|              Zambia|                  31|
|      Cayman Islands|                  31|
|          Costa Rica|                  31|
|              Jordan|                  31|
|              Sweden|                  32|
|             Namibia|                  

In [17]:
#2.Find average income of people who are born in USA
print("\n Average income of people who are born in USA")
ans=hc.sql("select avg(income) from tab where Birth_Country='United States of America'")
print(ans.show())


 Average income of people who are born in USA
+------------------+
|       avg(income)|
+------------------+
|208759.82352941178|
+------------------+

None


In [20]:
#3.How many people have income more than 100000 but loan is not approved
print("\n People with income more than 100000 but loan not approved")
ans=hc.sql("select count(SSN) from tab where income >= 100000 and Loan_Approved==False ")
print(ans.show())


 People with income more than 100000 but loan not approved
+----------+
|count(SSN)|
+----------+
|      4009|
+----------+

None


In [21]:
#4.Find top 10 people with highest income in USA
print('\n Top 10 people with highest income in USA')
ans=hc.sql("select First_Name,Last_Name from tab where Birth_Country='United States of America' order by Income desc LIMIT 10")
print(ans.show())


 Top 10 people with highest income in USA
+----------+---------+
|First_Name|Last_Name|
+----------+---------+
|    Alyssa|   Miller|
|    Hunter|    Walls|
|      Rose|Henderson|
|  Danielle|  Leonard|
|     Terry|    Klein|
|     Cindy|   Newton|
|     Scott| Mitchell|
|   Christy| Sandoval|
|     Kelly| Reynolds|
|  Kristina|    Smith|
+----------+---------+

None


In [22]:
#5.Number of distinct jobs
print('\n Number of distinct jobs')
ans=hc.sql('select count(distinct Job) from Tab')
print(ans.show())


 Number of distinct jobs
+-------------------+
|count(DISTINCT Job)|
+-------------------+
|                639|
+-------------------+

None


In [23]:
#6.How many writers earn less than 100000
print('\n Number of writers earn less than 100000')
ans=hc.sql('select count(SSN) from tab where Income<100000')
print(ans.show())


 Number of writers earn less than 100000
+----------+
|count(SSN)|
+----------+
|      2033|
+----------+

None


In [13]:
sc.stop()